In [ ]:
import os
import mlflow
import pandas as pd
import mysql.connector
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import mlflow.sklearn
from itertools import product
import pickle

In [2]:
# ---------------- Configuración de MLflow ----------------
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.101.195:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://10.43.101.195:5000")  # Verificar que esté en ejecución
mlflow.set_experiment("taller_grupo5")

# Configurar el autologging
mlflow.sklearn.autolog(log_model_signatures=True, log_input_examples=True, registered_model_name="RandomForestModel")
print("MLflow configurado correctamente")

2025/03/16 10:47:46 INFO mlflow.tracking.fluent: Experiment with name 'taller_grupo5' does not exist. Creating a new experiment.


MLflow configurado correctamente


In [3]:
# ---------------- Conexión a MySQL ----------------
DB_CONFIG = {
    "host": "10.43.101.195",  # ✅ Asegurar que es solo la IP
    "user": "admin",
    "password": "admingrupo5",
    "database": "data_db",
    "port": 3308  # ✅ Verificar el puerto correcto
}

try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Eliminar la tabla si ya existe
    cursor.execute("DROP TABLE IF EXISTS penguins;")

    # Crear la tabla
    cursor.execute('''
        CREATE TABLE penguins (
            id INT AUTO_INCREMENT PRIMARY KEY,
            species VARCHAR(50) NOT NULL,
            island VARCHAR(50) NOT NULL,
            culmen_length_mm FLOAT NOT NULL,
            culmen_depth_mm FLOAT NOT NULL,
            flipper_length_mm FLOAT NOT NULL,
            body_mass_g FLOAT NOT NULL,
            sex VARCHAR(10) NOT NULL
        );
    ''')

    conn.commit()
    print("Tabla 'penguins' creada exitosamente.")

    # Cerrar conexiones
    cursor.close()
    conn.close()

except mysql.connector.Error as err:
    print(f"Error al conectar a MySQL: {err}")

✔ Tabla 'penguins' creada exitosamente.


In [8]:
try:
    # Cargar el dataset desde CSV
    df = pd.read_csv("/work/data/penguins_size.csv").dropna()

    # Conexión a MySQL
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Consulta SQL optimizada para inserción masiva
    query = """
        INSERT INTO penguins (species, island, culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, sex)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """

    # Convertir el dataframe a una lista de tuplas para ejecutar con `executemany`
    data_tuples = list(df.itertuples(index=False, name=None))
    
    # Insertar los datos en una sola llamada para mejorar rendimiento
    cursor.executemany(query, data_tuples)

    # Confirmar cambios
    conn.commit()
    print(f"Datos cargados exitosamente. Registros insertados: {cursor.rowcount}")

    # Cerrar conexiones
    cursor.close()
    conn.close()

except mysql.connector.Error as err:
    print(f"Error al conectar a MySQL: {err}")

except FileNotFoundError:
    print("Error: El archivo CSV no fue encontrado.")

Datos cargados exitosamente. Registros insertados: 334


In [10]:
# ---------------- Preprocesamiento de datos en MySQL ----------------
try:
    # Conectar a MySQL
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Extraer los datos de la tabla original
    cursor.execute("SELECT id, species, island, culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, sex FROM penguins")
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Preprocesamiento: eliminar valores nulos
    df.dropna(inplace=True)

    # Eliminar la tabla si ya existe
    cursor.execute("DROP TABLE IF EXISTS penguins_clean")

    # Crear la tabla limpia
    cursor.execute('''
        CREATE TABLE penguins_clean (
            id INT AUTO_INCREMENT PRIMARY KEY,
            species VARCHAR(50),
            island VARCHAR(50),
            culmen_length_mm FLOAT,
            culmen_depth_mm FLOAT,
            flipper_length_mm FLOAT,
            body_mass_g FLOAT,
            sex VARCHAR(10)
        );
    ''')

    # **Verificar qué columnas estamos insertando**
    df = df[['species', 'island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']]

    # **Convertir a tuplas**
    data = [tuple(row) for row in df.to_numpy()]

    # **Insertar los datos limpios con `executemany()`**
    query = """
        INSERT INTO penguins_clean (species, island, culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, sex)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """
    cursor.executemany(query, data)

    # Confirmar cambios
    conn.commit()
    print(f"Datos preprocesados exitosamente. Registros insertados: {cursor.rowcount}")

    # Cerrar conexiones
    cursor.close()
    conn.close()

except mysql.connector.Error as err:
    print(f"Error al conectar a MySQL: {err}")

✔ Datos preprocesados exitosamente. Registros insertados: 1002


In [21]:
#GridSearch

# ---------------- Conectar y obtener datos de MySQL ----------------
try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    
    # Obtener datos preprocesados
    cursor.execute("SELECT species, culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g FROM penguins_clean")
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Cerrar conexión a MySQL
    cursor.close()
    conn.close()

    print("Datos cargados exitosamente desde MySQL")

except mysql.connector.Error as err:
    print(f"Error al conectar a MySQL: {err}")
    exit()

# ---------------- Preparación de datos ----------------
X = df[['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']]
y = df['species']

# Convertir la variable objetivo a valores numéricos con LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Guardar el LabelEncoder para usarlo en la inferencia
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ---------------- Configuración de MLflow ----------------
mlflow.set_tracking_uri("http://10.43.101.195:5000")
mlflow.set_experiment("taller_grupo5")

# ---------------- Definir GridSearch con hiperparámetros ----------------
param_grid = {
    "n_estimators": [25, 50, 75, 100],
    "max_depth": [4, 6, 8, 10],
    "max_features": [2, 3, 4]
}

# Crear modelo base
rf = RandomForestClassifier(random_state=42)

# Aplicar GridSearchCV para encontrar la mejor combinación
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, verbose=2)

# Guardar el LabelEncoder para usarlo en la inferencia
label_encoder_path = "label_encoder.pkl"
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)
    
# ---------------- Ejecutar entrenamiento con MLflow ----------------
with mlflow.start_run(run_name="rf_grid_search_v5"):
    grid_search.fit(X_train, y_train)

    # Obtener mejor modelo
    best_model = grid_search.best_estimator_

    # Registrar hiperparámetros en MLflow
    mlflow.log_params(grid_search.best_params_)

    # Guardar el mejor modelo en MLflow
    mlflow.sklearn.log_model(best_model, "RandomForestModel")

    # Subir el LabelEncoder a MLflow
    mlflow.log_artifact(label_encoder_path)

    print(f"Mejor modelo encontrado con parámetros: {grid_search.best_params_}")

print("Todos los experimentos fueron registrados en MLflow.")


Datos cargados exitosamente desde MySQL
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] END .......max_depth=4, max_features=2, n_estimators=25; total time=   0.1s
[CV] END .......max_depth=4, max_features=2, n_estimators=25; total time=   0.1s
[CV] END .......max_depth=4, max_features=2, n_estimators=25; total time=   0.1s
[CV] END .......max_depth=4, max_features=2, n_estimators=50; total time=   0.1s
[CV] END .......max_depth=4, max_features=2, n_estimators=50; total time=   0.1s
[CV] END .......max_depth=4, max_features=2, n_estimators=50; total time=   0.1s
[CV] END .......max_depth=4, max_features=2, n_estimators=75; total time=   0.2s
[CV] END .......max_depth=4, max_features=2, n_estimators=75; total time=   0.2s
[CV] END .......max_depth=4, max_features=2, n_estimators=75; total time=   0.2s
[CV] END ......max_depth=4, max_features=2, n_estimators=100; total time=   0.3s
[CV] END ......max_depth=4, max_features=2, n_estimators=100; total time=   0.3s
[CV] EN

Registered model 'RandomForestModel' already exists. Creating a new version of this model...
2025/03/16 11:54:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestModel, version 42
Created version '42' of model 'RandomForestModel'.
2025/03/16 11:54:37 INFO mlflow.sklearn.utils: Logging the 5 best runs, 43 runs will be omitted.


Mejor modelo encontrado con parámetros: {'max_depth': 6, 'max_features': 2, 'n_estimators': 25}
Todos los experimentos fueron registrados en MLflow.
